<a href="https://colab.research.google.com/github/nidhisdata/Python-Learnings/blob/main/Project_1_DS_Course_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -L -o dataset.zip\
  https://www.kaggle.com/api/v1/datasets/download/meirnizri/covid19-dataset
! unzip dataset.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 4776k  100 4776k    0     0  5392k      0 --:--:-- --:--:-- --:--:--     0
Archive:  dataset.zip
  inflating: Covid Data.csv          


In [2]:
!pip install xgboost


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, r2_score, mean_squared_error
import numpy as np
import xgboost as xgb

In [4]:
df = pd.read_csv('/content/Covid Data.csv')
df.head()

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
0,2,1,1,1,03/05/2020,97,1,65,2,2,...,2,2,1,2,2,2,2,2,3,97
1,2,1,2,1,03/06/2020,97,1,72,97,2,...,2,2,1,2,2,1,1,2,5,97
2,2,1,2,2,09/06/2020,1,2,55,97,1,...,2,2,2,2,2,2,2,2,3,2
3,2,1,1,1,12/06/2020,97,2,53,2,2,...,2,2,2,2,2,2,2,2,7,97
4,2,1,2,1,21/06/2020,97,2,68,97,1,...,2,2,1,2,2,2,2,2,3,97


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   USMER                 1048575 non-null  int64 
 1   MEDICAL_UNIT          1048575 non-null  int64 
 2   SEX                   1048575 non-null  int64 
 3   PATIENT_TYPE          1048575 non-null  int64 
 4   DATE_DIED             1048575 non-null  object
 5   INTUBED               1048575 non-null  int64 
 6   PNEUMONIA             1048575 non-null  int64 
 7   AGE                   1048575 non-null  int64 
 8   PREGNANT              1048575 non-null  int64 
 9   DIABETES              1048575 non-null  int64 
 10  COPD                  1048575 non-null  int64 
 11  ASTHMA                1048575 non-null  int64 
 12  INMSUPR               1048575 non-null  int64 
 13  HIPERTENSION          1048575 non-null  int64 
 14  OTHER_DISEASE         1048575 non-null  int64 
 15

In [6]:
df['DATE_DIED'].head()

,DATE_DIED
0,03/05/2020
1,03/06/2020
2,09/06/2020
3,12/06/2020
4,21/06/2020


In [7]:
df['DATE_DIED'] = ['01/01/2020' if row =='9999-99-99' else row for row in df['DATE_DIED']]

In [8]:
# cleaning process
# better drop columns ->  INTUBED, PREGNANT instead of dropping rows
df.replace([97,99], pd.NA, inplace=True)
display(df.isnull().sum())
df.dropna(inplace=True)
display(df.isnull().sum())

,0
USMER,0
MEDICAL_UNIT,0
SEX,0
PATIENT_TYPE,0
DATE_DIED,0
INTUBED,855869
PNEUMONIA,16003
AGE,221
PREGNANT,523511
DIABETES,0


,0
USMER,0
MEDICAL_UNIT,0
SEX,0
PATIENT_TYPE,0
DATE_DIED,0
INTUBED,0
PNEUMONIA,0
AGE,0
PREGNANT,0
DIABETES,0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 78120 entries, 5 to 1048569
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   USMER                 78120 non-null  int64 
 1   MEDICAL_UNIT          78120 non-null  int64 
 2   SEX                   78120 non-null  int64 
 3   PATIENT_TYPE          78120 non-null  int64 
 4   DATE_DIED             78120 non-null  object
 5   INTUBED               78120 non-null  object
 6   PNEUMONIA             78120 non-null  object
 7   AGE                   78120 non-null  object
 8   PREGNANT              78120 non-null  object
 9   DIABETES              78120 non-null  int64 
 10  COPD                  78120 non-null  int64 
 11  ASTHMA                78120 non-null  int64 
 12  INMSUPR               78120 non-null  int64 
 13  HIPERTENSION          78120 non-null  int64 
 14  OTHER_DISEASE         78120 non-null  int64 
 15  CARDIOVASCULAR        78120 non-null  i

In [13]:
df['DATE_DIED'] = pd.to_datetime(df['DATE_DIED'],format = '%d/%m/%Y')

In [14]:
df['day'] = df['DATE_DIED'].dt.day
df['month'] = df['DATE_DIED'].dt.month
df['year'] = df['DATE_DIED'].dt.year
display(df)

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU,day,month,year
5,2,1,1,2,2020-01-01,2,1,40,2,2,...,2,2,2,2,2,3,2,1,1,2020
8,2,1,1,2,2020-01-01,2,2,37,2,1,...,2,2,1,2,2,3,2,1,1,2020
9,2,1,1,2,2020-01-01,2,2,25,2,2,...,2,2,2,2,2,3,2,1,1,2020
16,2,1,1,2,2020-01-01,2,1,80,2,2,...,2,2,2,2,2,3,1,1,1,2020
56,2,1,1,2,2020-01-01,1,1,58,2,2,...,2,1,1,2,2,7,1,1,1,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048535,1,13,1,2,2020-01-01,2,2,79,2,2,...,2,2,2,2,2,7,2,1,1,2020
1048553,2,13,1,2,2020-01-01,2,1,61,2,1,...,2,2,2,2,2,7,2,1,1,2020
1048554,2,13,1,2,2020-01-01,2,1,63,2,2,...,2,2,1,2,2,7,2,1,1,2020
1048567,1,13,1,2,2020-01-01,2,2,23,2,1,...,2,2,1,2,2,7,2,1,1,2020


### CLASSIFICATION PART

In [15]:
df_classifcation = df.drop(columns=['DATE_DIED'])
X = df_classifcation.drop(columns=['CLASIFFICATION_FINAL'])
y = df_classifcation['CLASIFFICATION_FINAL']

In [16]:
X.columns

Index(['USMER', 'MEDICAL_UNIT', 'SEX', 'PATIENT_TYPE', 'INTUBED', 'PNEUMONIA',
       'AGE', 'PREGNANT', 'DIABETES', 'COPD', 'ASTHMA', 'INMSUPR',
       'HIPERTENSION', 'OTHER_DISEASE', 'CARDIOVASCULAR', 'OBESITY',
       'RENAL_CHRONIC', 'TOBACCO', 'ICU', 'day', 'month', 'year'],
      dtype='object')

In [17]:
preprocessor = LabelEncoder()
for col in X.columns:
  X[col] = preprocessor.fit_transform(X[col])

display(X.head())

scaler = StandardScaler()
X = scaler.fit_transform(X)
X.shape

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,COPD,...,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,ICU,day,month,year
5,1,0,0,0,1,0,40,1,1,1,...,1,1,1,1,1,1,1,0,0,0
8,1,0,0,0,1,1,37,1,0,1,...,0,1,1,0,1,1,1,0,0,0
9,1,0,0,0,1,1,25,1,1,1,...,1,1,1,1,1,1,1,0,0,0
16,1,0,0,0,1,0,80,1,1,1,...,0,1,1,1,1,1,0,0,0,0
56,1,0,0,0,0,0,58,1,1,1,...,0,1,0,0,1,1,0,0,0,0


(78120, 22)

classification: covid test findings. Values 1-3 mean that the patient was diagnosed with covid in different degrees. 4 or higher means that the patient is not a carrier of covid or that the test is inconclusive.

In [18]:
y = y.apply(lambda x: 1 if x >= 4 else 0)
display(y.head())

y = np.array(list(y))
y.shape

,CLASIFFICATION_FINAL
5,0
8,0
9,0
16,0
56,1


(78120,)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(62496, 22)
(62496,)
(15624, 22)
(15624,)


Model 1 - Classification

In [20]:
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)


print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.64      0.68      0.66      8371
           1       0.60      0.55      0.58      7253

    accuracy                           0.62     15624
   macro avg       0.62      0.62      0.62     15624
weighted avg       0.62      0.62      0.62     15624



Model 2 - Classification (Logistic Regression TODO)

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.65      0.67      0.66      8371
           1       0.61      0.59      0.60      7253

    accuracy                           0.63     15624
   macro avg       0.63      0.63      0.63     15624
weighted avg       0.63      0.63      0.63     15624



### Preparing Data for Regressor

In [22]:
df_regressor = df.drop(columns=['DATE_DIED'])
X = df_regressor.drop(columns=['AGE']) # features
y = df_regressor['AGE'] # prediction

X['CLASIFFICATION_FINAL'] = X['CLASIFFICATION_FINAL'].apply(lambda x: 1 if x >= 4 else 0)

preprocessor = LabelEncoder()
for col in X.columns:
  X[col] = preprocessor.fit_transform(X[col])

display(X.head())

scaler = StandardScaler()
X = scaler.fit_transform(X)
X.shape

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,INTUBED,PNEUMONIA,PREGNANT,DIABETES,COPD,ASTHMA,...,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU,day,month,year
5,1,0,0,0,1,0,1,1,1,1,...,1,1,1,1,1,0,1,0,0,0
8,1,0,0,0,1,1,1,0,1,1,...,1,1,0,1,1,0,1,0,0,0
9,1,0,0,0,1,1,1,1,1,1,...,1,1,1,1,1,0,1,0,0,0
16,1,0,0,0,1,0,1,1,1,1,...,1,1,1,1,1,0,0,0,0,0
56,1,0,0,0,0,0,1,1,1,1,...,1,0,0,1,1,1,0,0,0,0


(78120, 22)

In [23]:
scaler = StandardScaler()
y = scaler.fit_transform(np.array(y).reshape(-1, 1))
y.shape

(78120, 1)

In [24]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(62496, 22)
(62496, 1)
(15624, 22)
(15624, 1)


MODEL 3 - RandomForestRegressor

In [25]:
regressor1 = RandomForestRegressor(random_state=42)
regressor1.fit(X_train, y_train)
y_pred = regressor1.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost Regressor - MSE: {mse}, R2: {r2}")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


XGBoost Regressor - MSE: 0.6775932960530275, R2: 0.3258837234547579


Model 2 - Regressor (TODO)

In [26]:
xgb_model = xgb.XGBRegressor(
    n_estimators=100,       # Number of boosting rounds
    learning_rate=0.1,      # Step size shrinkage to prevent overfitting
    max_depth=3,            # Maximum depth of the tree
    subsample=0.8,          # Fraction of samples used for training
    colsample_bytree=0.8,   # Fraction of features used for training
    random_state=42         # Random seed for reproducibility
)
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Evaluate the model using Mean Squared Error (MSE) and R² Score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the results
print(f"XGBoost Regressor - MSE: {mse}, R2: {r2}")

XGBoost Regressor - MSE: 0.6417095375664417, R2: 0.36158334710855067
